In [1]:
# Setup
! pip install seqeval evaluate
! pip install --upgrade nevergrad # upgrade to ensure latest version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=3065d8b1710b7be76c237cbe86825e39c0e09d1a1fe05923fa8f1d0e23f40cac
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.0/498.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 25.0 MB/s e

In [2]:
# Library imports
from transformers import AutoTokenizer, AutoModel, pipeline, AutoConfig, DistilBertForSequenceClassification, DistilBertModel, DistilBertConfig, DistilBertPreTrainedModel, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.tokenization_utils_base import BatchEncoding
from datasets import Dataset, DatasetDict
import torch
import torch.nn as nn
from google.colab import drive, userdata
import pickle
import random
import re
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.express as px
import evaluate
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import re
from tqdm import tqdm
import torch.nn.functional as F
from torch.utils.data import DataLoader
import nevergrad as ng
import wandb
import shutil
import tempfile
import os

In [3]:
# Mount drive
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files


In [4]:
# View all pandas columns, rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# Define file read function
def read_pickle(dict_file):
  with open(dict_file, 'rb') as file:
    return pickle.load(file)

In [6]:
# Load custom trained model

checkpoint = "Heather-Driver/distilbert-NER-LinearAlg-finetuned"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, do_lower_case=False)
distilbert_model = DistilBertModel.from_pretrained(checkpoint)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [7]:
def extract_window(sentence, predicate, window_size):
  """This function creates a window around the matching predicate in order to tokenize and later get the span vectors for the window.
  The function adjusts according to the window size wanted"""
  tokens = sentence.split()

  # Find the starting index of the predicate in the sentence (find all word positions for the predicate)
  pattern = re.escape(predicate)  # Escape the predicate string to handle special characters if any
  match = re.search(pattern, sentence)

  if not match:
      return "Predicate not found in the sentence."

  # Get the index of where the predicate starts in the list of tokens
  start_index = len(sentence[:match.start()].split())  # Token index of the start of predicate

  # Define the sample window
  start_window = max(0, start_index - window_size)
  end_window = min(len(tokens), start_index + len(predicate.split()) + window_size)

  # Create the window of words around the predicate
  window = tokens[start_window:end_window]

  # If the window is too short at the beginning or the end, adjust to take as many as possible
  if start_window == 0:
      # If the window is at the start, extend the end if possible
      end_window = min(len(tokens), start_index + len(predicate.split()) + window_size)
  if end_window == len(tokens):
      # If the window is at the end, extend the start if possible
      start_window = max(0, start_index - window_size)

  # Create the window of words around the predicate again after adjustments
  window = tokens[start_window:end_window]
  return ' '.join(window)

def adds_context_window(window_size, df):
  for i in range(len(df)):
    text = extract_window(df.at[i, 'sentence'], df.at[i, 'predicate'], window_size=window_size)
    df.at[i, 'context_window'] = text
  return df

In [8]:
# Read in dictionary
predicate_data = read_pickle('predicate_data.pkl')
predicate_data = predicate_data.rename(columns={'Window_1': 'context_window', 'Label': 'string_label'})
predicate_data.columns = predicate_data.columns.str.lower()

In [9]:
# Need mapping of classification tags to their indices for model to use

index2tag = {idx:tag for idx, tag in enumerate(predicate_data['string_label'].unique())} # This is just a nonsignificant arbitrary mapping of the label to a number for training the model
tag2index = {tag:idx for idx, tag in enumerate(predicate_data['string_label'].unique())} # To lookup indices from tags

In [10]:
predicate_data['label'] = predicate_data['string_label'].map(tag2index)

In [11]:
predicate_data = adds_context_window(window_size=2, df=predicate_data)

In [12]:
predicate_data.head(2)

,sentence,subject,predicate,object,string_label,context_window,label
0,The Wishart distribution is used in multivaria...,wishart distribution,is used in,multivariate statistics,used in,Wishart distribution is used in multivariate s...,0
1,The Square Root Method is transformed by the a...,Square Root Method,transformed by,the application of inverse operations to deriv...,computation,Method is transformed by the application,1


In [13]:
X_train_indices, X_test_indices, y_train_indices, y_test_indices = train_test_split(predicate_data.index.to_numpy(), predicate_data['label'].to_numpy(),
                                                                                    test_size=0.05, random_state=42, stratify=predicate_data['label'].to_numpy())

# Repeat to get validation sub-sample of Train
X_train_indices, X_valid_indices, y_train_indices, y_valid_indices = train_test_split(X_train_indices, y_train_indices, test_size=0.3, random_state=42, stratify=y_train_indices)

## Preprocessing

In [14]:
dataset = Dataset.from_pandas(predicate_data[['sentence', 'label', 'context_window', 'predicate']])

In [15]:
# Select subsets of the dataset for train, test and validation
train_split = dataset.select(X_train_indices)
test_split = dataset.select(X_test_indices)
valid_split = dataset.select(X_valid_indices)

dataset = DatasetDict({
    'train': train_split,
    'test': test_split,
    'validation': valid_split
})

In [16]:
def preprocess_function_predicate(examples):
  context_inputs = tokenizer(examples["predicate"], return_tensors="pt", add_special_tokens=False, truncation=True, padding="max_length", max_length=256)
  return context_inputs

dataset = dataset.map(preprocess_function_predicate, batched=True)
# Rename the 'attention_mask' column to 'context_attention_mask'
dataset = dataset.rename_columns({"attention_mask": "predicate_attention_mask", "input_ids": "predicate_input_ids"})

Map:   0%|          | 0/292 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

In [17]:
def preprocess_function_context(examples):
  context_inputs = tokenizer(examples["context_window"], return_tensors="pt", add_special_tokens=True, truncation=True, padding="max_length", max_length=256)
  return context_inputs

dataset = dataset.map(preprocess_function_context, batched=True)
# Rename the 'attention_mask' column to 'context_attention_mask'
dataset = dataset.rename_columns({"attention_mask": "context_attention_mask", "input_ids": "context_input_ids"})

Map:   0%|          | 0/292 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

In [18]:
def preprocess_function(examples):
  inputs = tokenizer(examples["sentence"], return_tensors="pt", add_special_tokens=True, truncation=True, padding="max_length", max_length=256)
  return inputs

dataset = dataset.map(preprocess_function, batched=True)

dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label', 'context_attention_mask', 'context_input_ids', 'predicate_attention_mask', 'predicate_input_ids'])

Map:   0%|          | 0/292 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

## Developing the Model Parameters

In [19]:
class StandardDistilBertClassifier(DistilBertPreTrainedModel):
  config_class = DistilBertConfig

  def __init__(self, config):
    super().__init__(config)
    self.num_labels = config.num_labels
    # Model body
    self.distilbert = distilbert_model
    # Classification head
    self.classifier = nn.Linear(config.hidden_size, self.num_labels)  # Span * 3 (CLS, span, width)

  def _cls_embeddings(self, input_ids, attention_mask):
    outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
    embeddings = outputs.last_hidden_state # shape [1, 512, 768]
    cls_embedding = embeddings[:, 0:1, :].squeeze(1)  # squeeze converts [1, 1, 768] to [1, 768]
    return cls_embedding

  def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
    # Get CLS token embedding
    cls_embedding = self._cls_embeddings(input_ids=input_ids, attention_mask=attention_mask)
    # Classifier on cls_embedding
    logits = self.classifier(cls_embedding)
    # Loss calc
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    return SequenceClassifierOutput(loss=loss, logits=logits)

In [20]:
config = DistilBertConfig.from_pretrained("Heather-Driver/distilbert-NER-LinearAlg-finetuned")
config.label2id = tag2index
config.id2label = index2tag
config.num_labels = len(index2tag)

model = StandardDistilBertClassifier(config)
model.to(device)
model.gradient_checkpointing_enable()

In [21]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    return_tensors="pt",
)

In [22]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted") #y_true, y_pred
  acc = accuracy_score(labels, preds)
  precision = precision_score(labels, preds, average="weighted")
  recall = recall_score(labels, preds, average="weighted")
  return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

## Nevergrad optimisation

In [23]:
# Load the best hyperparameters from the pickle file
with open("best_hyperparameters.pkl", "rb") as f:
    best_params = pickle.load(f)

print("Loaded best hyperparameters:", best_params)

# You can now use these hyperparameters, for example:
# train_and_evaluate(**best_hyperparameters, name="final_model_run")

Loaded best hyperparameters: {'learning_rate': 5.314885705504048e-06, 'batch_size': 32, 'weight_decay': 7.395196046678932e-05, 'warmup_steps': 100, 'gradient_accumulation_steps': 1}


In [24]:
%cd '/content/drive/MyDrive/Colab Notebooks/Math_Graph/Model/transformers_cache'

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Math_Graph/Model/transformers_cache'
/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files


In [25]:
  # Set output_dir in TrainingArguments to this temporary folder.
  # After exiting the with-block, the directory is deleted.
os.environ["TRANSFORMERS_CACHE"] = '/content/drive/MyDrive/Colab Notebooks/Math_Graph/Model/transformers_cache'

In [27]:
def train_and_evaluate(learning_rate, batch_size, weight_decay, warmup_steps, gradient_accumulation_steps, name):
  """Function to train the model with different hyperparameters"""
  output_dir = '/content/drive/MyDrive/Colab Notebooks/Math_Graph/Model/' + name
  training_arguments = TrainingArguments(
      output_dir=output_dir,
      run_name=name,
      log_level="error",
      num_train_epochs=25,  # Keeping epochs fixed for now
      per_device_train_batch_size=int(batch_size),
      per_device_eval_batch_size=int(batch_size),
      learning_rate=learning_rate,
      weight_decay=weight_decay,
      warmup_steps=int(warmup_steps),
      gradient_accumulation_steps=int(gradient_accumulation_steps),
      eval_strategy="no",
      save_strategy="no",
      disable_tqdm=True,  # Avoid flooding the output
      report_to=["wandb"],
      load_best_model_at_end=True,
      push_to_hub=False,
      greater_is_better=True,
      save_safetensors=True,
      save_total_limit=1,
  )

  trainer = Trainer(
      model=model,
      args=training_arguments,
      train_dataset=dataset["train"],
      eval_dataset=dataset["validation"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
  )

  # Train and evaluate
  trainer.train()
  metrics = trainer.evaluate()

  # Finish the current W&B run so that a new run is created next time
  wandb.finish()

  # Delete the output directory to free disk space
  shutil.rmtree(output_dir, ignore_errors=True)

  # Return negative F1-score (Nevergrad minimizes)
  return -metrics["eval_f1"]

# Define the hyperparameter search space
instrum = ng.p.Instrumentation(
    learning_rate=ng.p.Log(lower=1e-6, upper=1e-3),        # Logarithmic scale
    batch_size=ng.p.TransitionChoice([4, 8, 16, 32]),
    weight_decay=ng.p.Log(lower=1e-5, upper=1e-1),
    warmup_steps=ng.p.TransitionChoice([100, 500, 1000, 2000]),
    gradient_accumulation_steps=ng.p.TransitionChoice([1, 2, 4])
)

# Initialize the optimizer with the instrumentation and a budget
optimizer = ng.optimizers.OnePlusOne(instrum, budget=20)  # 20 iterations

# Define a wrapper function that takes a dictionary of parameters and unpacks them for train_and_evaluate
# Wrapper function with dynamic naming
def objective_function_wrapper(**parameters):
    # Construct a unique name from the hyperparameters
    combo_name = (
        f"lr_{parameters['learning_rate']:.1e}_"
        f"bs_{parameters['batch_size']}_"
        f"wd_{parameters['weight_decay']:.1e}_"
        f"ws_{parameters['warmup_steps']}_"
        f"gas_{parameters['gradient_accumulation_steps']}"
    )
    combo_name = "optimized_model_baseline_" + combo_name
    return train_and_evaluate(**parameters, name=combo_name)

# Update the optimizer.minimize call to use the wrapper function
recommendation = optimizer.minimize(objective_function_wrapper)

# Print the best hyperparameters found
print("Best hyperparameters:", recommendation.kwargs)

# Save best hyperparameters to a pickle file
with open("best_hyperparameters.pkl", "wb") as f:
    pickle.dump(recommendation.kwargs, f)

print("Best hyperparameters:", recommendation.kwargs)
best_params = recommendation.kwargs

<ipython-input-27-7414e5df219e>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: heather-rink (h-driver) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

<ipython-input-27-7414e5df219e>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'train_runtime': 31.4092, 'train_samples_per_second': 232.416, 'train_steps_per_second': 1.592, 'train_loss': 8.091834716796875, 'epoch': 16.8}
{'eval_loss': 2.399357795715332, 'eval_accuracy': 0.0873015873015873, 'eval_f1': 0.014019232997335188, 'eval_precision': 0.007621567145376668, 'eval_recall': 0.0873015873015873, 'eval_runtime': 0.2072, 'eval_samples_per_second': 608.241, 'eval_steps_per_second': 19.309, 'epoch': 16.8}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁
train/global_step,▁▁
eval/accuracy,0.0873


Best hyperparameters: {'learning_rate': 1.0000000000000008e-06, 'batch_size': 32, 'weight_decay': 1.0000000000000008e-05, 'warmup_steps': 2000, 'gradient_accumulation_steps': 1}
Best hyperparameters: {'learning_rate': 1.0000000000000008e-06, 'batch_size': 32, 'weight_decay': 1.0000000000000008e-05, 'warmup_steps': 2000, 'gradient_accumulation_steps': 1}
